### 1. Scarica il file di embedding e importa i package

In [ ]:
!wget https://www.dropbox.com/s/b3jbd1bgf93rkw6/glove.6B.50d.txt

In [ ]:
import  os
import  numpy                    as np
from    tensorflow.python.keras  import losses

### 2. Leggi il file di embedding

In [ ]:
def read_glove( glove_file ):
    embedding   = {}
    
    # legge il file GloVe riga per riga
    with open( glove_file, 'r' ) as f:
        cnt = 0
        
        for l in f:
            # per ogni riga del file, estrae la parola e il vettore
            word, vector        = l.split( maxsplit=1 )
            vector              = np.matrix( vector ).A1 
            # ...e li salva nel dizionario
            embedding[ word ]   = vector
            
            # stampa un messaggio di conferma
            cnt += 1
            if not cnt % 10000:
                print( f"read {cnt:,} of 400,000 words" )

    print( "Done!" )
    return embedding

In [ ]:
glove_file  = "glove.6B.50d.txt"
embedding   = read_glove( glove_file )

### 3. Gioca con gli embedding

In [ ]:
def embed( word ):
    if isinstance( word, str ):
        # controlla se la parola è presente nel dizionario
        if word not in embedding.keys():
            return False
        return embedding[ word ]
    return word

In [ ]:
print( embed( "unicorn" ) )

In [ ]:
def sim( word1, word2 ):
    word1   = embed( word1 )
    word2   = embed( word2 )
    s       = losses.cosine_similarity( word1, word2 )
    return s.numpy()

In [ ]:
sim( 'dog', 'wolf' )

In [ ]:
sim( 'dog', 'galaxy' )

### 4. Trova le parole più simili

In [ ]:
def closest( word, n_words=5, limit=50000 ):
    word      = embed( word )
    cnt       = 0
    
    # mantiene le parole con similarità migliore
    best      = [ ( None, 1.0 ) ]
    
    # analizza tutto il dizionario
    for w in embedding.keys():
    
        # per ogni parola, misura la similarità con quella in input
        score       = sim( embedding[ w ], word )

        # salta la parola se è identica a quella in input
        if ( score + 1 ) < 0.05:
            continue

        # salva la parola se migliore di una di quelle salvate finora
        for i, ( v, s ) in enumerate( best ):
            if score < s:
                best.insert( i, ( w, score ) )
                del best[ n_words: ]
                break

        # visualizza il progresso della funzione
        cnt += 1
        if not cnt % 1000:
            print( f"checked {cnt:,} of {limit:,} words" )
        if cnt > limit:
            print()
            break

    return best

In [ ]:
closest( 'queen', n_words=10, limit=20000 )

### 5. Somma le parole insieme

In [ ]:
# somma tra due parole
def plus( word1, word2 ):
    word1   = embed( word1 )
    word2   = embed( word2 )
    return word1 + word2

# sottrazione tra due parole
def minus( word1, word2 ):
    word1   = embed( word1 )
    word2   = embed( word2 )
    return word1 - word2

In [ ]:
w = minus( 'king', 'man' )
w = plus( w, 'woman' )
sim( w, 'queen' )

In [ ]:
w = minus( 'queen', 'throne' )
w = plus( w, 'job' )
closest( w, n_words=30, limit=30000 )